# Setup

In [56]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from datetime import datetime
import multiprocessing as mp
import multi_task
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [263]:
scorers = pd.read_csv('../dat/goalscorers.csv')
results = pd.read_csv('../dat/results.csv')
shootouts = pd.read_csv('../dat/shootouts.csv')

In [264]:
# Hard Codeing some problematic names

results.home_team.loc[results['home_team'] == 'China PR'] = 'China'
results.home_team.loc[results['home_team'] == 'DR Congo'] = 'Dem. Rep. of Congo'
results.home_team.loc[results['home_team'] == 'Republic of Ireland'] = 'Ireland'
results.home_team.loc[results['home_team'] == 'São Tomé and Príncipe'] = 'São Tomé e Príncipe'
results.home_team.loc[results['home_team'] == 'North Macedonia'] = 'Macedonia'


results.away_team.loc[results['away_team'] == 'China PR'] = 'China'
results.away_team.loc[results['away_team'] == 'DR Congo'] = 'Dem. Rep. of Congo'
results.away_team.loc[results['away_team'] == 'Republic of Ireland'] = 'Ireland'
results.away_team.loc[results['away_team'] == 'São Tomé and Príncipe'] = 'São Tomé e Príncipe'
results.away_team.loc[results['away_team'] == 'North Macedonia'] = 'Macedonia'

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [269]:
resultswc = results.loc[(results['date'] >= '2022-11-19')]
results = results.dropna().reset_index(drop=True)
resultswc

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44104,2022-11-20,Qatar,Ecuador,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False
44105,2022-11-21,Senegal,Netherlands,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44106,2022-11-21,England,Iran,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44107,2022-11-21,United States,Wales,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44108,2022-11-22,Argentina,Saudi Arabia,NaN,NaN,FIFA World Cup,Lusail,Qatar,True
44109,2022-11-22,Mexico,Poland,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44110,2022-11-22,Denmark,Tunisia,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44111,2022-11-22,France,Australia,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True
44112,2022-11-23,Germany,Japan,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44113,2022-11-23,Spain,Costa Rica,NaN,NaN,FIFA World Cup,Doha,Qatar,True


In [235]:
all_teams = list(set( list(results.home_team.unique()) + list(results.away_team.unique())))

In [236]:
results2 = results.copy()

In [237]:
results =results.dropna(axis=0)
results = results.dropna().reset_index(drop=True)
results = results.drop(results[results.tournament == 'Friendly'].index)
results = results.dropna().reset_index(drop=True)
results['date'] = pd.to_datetime(results['date'])
results = results.loc[(results['date'] >= '2012-10-10')]
results = results.dropna().reset_index(drop=True)

In [238]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2012-10-10,French Guiana,Trinidad and Tobago,1.0,4.0,CFU Caribbean Cup qualification,Basseterre,Saint Kitts and Nevis,True
1,2012-10-11,Brunei,Laos,1.0,3.0,AFF Championship,Yangon,Myanmar,True
2,2012-10-11,Myanmar,Cambodia,3.0,0.0,AFF Championship,Yangon,Myanmar,False
3,2012-10-12,Albania,Iceland,1.0,2.0,FIFA World Cup qualification,Tirana,Albania,False
4,2012-10-12,Antigua and Barbuda,United States,1.0,2.0,FIFA World Cup qualification,St. John's,Antigua and Barbuda,False
...,...,...,...,...,...,...,...,...,...
6348,2022-09-30,Fiji,Solomon Islands,1.0,0.0,MSG Prime Minister's Cup,Luganville,Vanuatu,True
6349,2022-11-05,Brunei,Timor-Leste,6.0,2.0,AFF Championship qualification,Bandar Seri Begawan,Brunei,False
6350,2022-11-08,Brunei,Timor-Leste,0.0,1.0,AFF Championship qualification,Bandar Seri Begawan,Brunei,False
6351,2022-11-16,Lithuania,Iceland,0.0,0.0,Baltic Cup,Kaunas,Lithuania,False


In [8]:
results2 = results2.loc[(results2['date'] >= '2022-10-10')]
results2 = results2.loc[(results2['tournament'] == 'FIFA World Cup')]

In [9]:
teams = list(results2.home_team) + list(results2.away_team)
teams = list(set(teams))

In [10]:
results = results.loc[(results['home_team'].isin(teams) | results['away_team'].isin(teams))]

In [11]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
4,2012-10-12,Antigua and Barbuda,United States,1.0,2.0,FIFA World Cup qualification,St. John's,Antigua and Barbuda,False
5,2012-10-12,Argentina,Uruguay,3.0,0.0,FIFA World Cup qualification,Mendoza,Argentina,False
7,2012-10-12,Belarus,Spain,0.0,4.0,FIFA World Cup qualification,Minsk,Belarus,False
9,2012-10-12,Bulgaria,Denmark,1.0,1.0,FIFA World Cup qualification,Sofia,Bulgaria,False
10,2012-10-12,Canada,Cuba,3.0,0.0,FIFA World Cup qualification,Toronto,Canada,False
...,...,...,...,...,...,...,...,...,...
6330,2022-09-26,England,Germany,3.0,3.0,UEFA Nations League,London,England,False
6338,2022-09-27,Ecuador,Japan,0.0,0.0,Kirin Challenge Cup,Düsseldorf,Germany,True
6339,2022-09-27,Portugal,Spain,0.0,1.0,UEFA Nations League,Braga,Portugal,False
6340,2022-09-27,Switzerland,Czech Republic,2.0,1.0,UEFA Nations League,St. Gallen,Switzerland,False


## WebScrape ELO ratings

In [12]:
from itertools import repeat

In [13]:
years = range(2012, 2023)
ratings_by_year = [{} for i in repeat(None, 11)]

In [14]:
years = range(2012, 2023)
for idx, year in enumerate(years):
    url = f'https://www.international-football.net/elo-ratings-table?year={year}&month=06&day=01&confed=&old-team=#selection'
    page = requests.get(url) # Getting page HTML through request
    soup = BeautifulSoup(page.content, 'html.parser') # Parsing content using beautifulsoup
    trim = soup.text[soup.text.find('As calculated on website www.eloratings.net'):]
    for team in all_teams:
        team_name = '.' + team
        counter = 0
        if team_name in trim:
            try:
                ratings_by_year[idx][team] = int(trim[trim.find(team) + len(team): trim.find(team) + len(team) + 4])
            except Exception:
                while counter < 10:
                    updated_trim = trim[trim.find(team) + len(team):]
                    try:
                        ratings_by_year[idx][team] = int(updated_trim[updated_trim.find(team) + len(team): updated_trim.find(team) + len(team) + 4])
                        break
                    except Exception:
                        counter += 1

In [15]:
#results = results.reset_index()  # make sure indexes pair with number of rows
results['home_rating'] = 0
results['away_rating'] = 0

<ipython-input-15-38d288d72901>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['home_rating'] = 0
<ipython-input-15-38d288d72901>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['away_rating'] = 0


In [16]:
results.home_rating[4]

0

In [17]:
results = results.dropna().reset_index(drop=True)

for idx, row in results.iterrows():
    item = row.date.year - 2012
    try:
        rat = ratings_by_year[item][row.home_team]
        if rat > 3000:
            rat = int(str(rat)[:-1])
        results.home_rating[idx] = rat
    except Exception:
        if row.home_team == 'Macedonia':
            results.home_rating[idx] = 1518
        elif row.home_team == 'United States Virgin Islands':
            results.home_rating[idx] = 680
        elif row.home_team == 'Taiwan':
            results.home_rating[idx] = 950
    try:
        rat = ratings_by_year[item][row.away_team]
        if rat > 3000:
            rat = int(str(rat)[:-1])
        results.away_rating[idx] = rat
    except Exception:
        if row.away_team == 'Macedonia':
            results.home_rating[idx] = 1518
        elif row.away_team == 'United States Virgin Islands':
            results.home_rating[idx] = 680
        elif row.away_team == 'Taiwan':
            results.home_rating[idx] = 950

<ipython-input-17-cbc31864b1d5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.home_rating[idx] = rat
<ipython-input-17-cbc31864b1d5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.away_rating[idx] = rat
<ipython-input-17-cbc31864b1d5>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.home_rating[idx] = 950
<ipython-input-17-cbc31864b1d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [18]:
results['long'] = 0.0
results['lat'] = 0.0

In [19]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

In [21]:
results = results.dropna().reset_index(drop=True)
# introduce parralel programming if run again.
for idx, row in results.iterrows():
    try:
        location = geolocator.geocode(row.city)[-1]
        long =  location[0]
        lat =  location[1]
        results.long[idx] = long
        results.lat[idx] = lat
    except Exception:
        try: 
            location = geolocator.geocode(row.country)[-1]
            long =  location[0]
            lat =  location[1]
            results.long[idx] = long
            results.lat[idx] = lat
        except Exception:
            pass

<ipython-input-21-7d1b310cabb2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.long[idx] = long
<ipython-input-21-7d1b310cabb2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.lat[idx] = lat


In [655]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44099,2022-11-16,Turkey,Scotland,2.0,1.0,Friendly,Diyarbakir,Turkey,False
44100,2022-11-16,United Arab Emirates,Argentina,0.0,5.0,Friendly,Abu Dhabi,United Arab Emirates,False
44101,2022-11-16,Uzbekistan,Kazakhstan,2.0,0.0,Friendly,Tashkent,Uzbekistan,False
44102,2022-11-16,Lithuania,Iceland,0.0,0.0,Baltic Cup,Kaunas,Lithuania,False


results.to_csv(r'../dat/results_edit.csv', index=False)

In [1921]:
results = pd.read_csv('../dat/results_edit.csv')

In [1922]:
results['temp'] = np.nan
results['prcp'] = np.nan
results['snow'] = np.nan
results['wspd'] = np.nan
results['pres'] = np.nan

In [1923]:
results.loc[0]

date                             2012-10-12
home_team               Antigua and Barbuda
away_team                     United States
home_score                                1
away_score                                2
tournament     FIFA World Cup qualification
city                             St. John's
country                 Antigua and Barbuda
neutral                               False
home_rating                            1284
away_rating                            1752
long                                47.5615
lat                                -52.7126
temp                                    NaN
prcp                                    NaN
snow                                    NaN
wspd                                    NaN
pres                                    NaN
Name: 0, dtype: object

In [1924]:
results['date'] = pd.to_datetime(results['date'])
pool = mp.Pool(processes=mp.cpu_count())
new_rows = pool.map(multi_task.task, [row for row in results.iterrows()])
weather_results = pd.DataFrame.from_records(new_rows, columns = list(results.keys()))

In [1925]:
weather_results.drop(weather_results[weather_results['tournament'] == 'UEFA Nations League'].index, inplace = True)

In [1926]:
train = weather_results.drop(['city', 'long', 'lat', 'tournament', 'country'], axis=1)

In [1927]:
train.home_team = train.home_team.astype("category")
train.away_team = train.away_team.astype("category")
#train.country = train.country.astype("category")

train.home_rating = train.home_rating.astype("int32")
train.away_rating = train.away_rating.astype("int32")
train.home_score = train.home_score.astype("int32")
train.away_score = train.away_score.astype("int32")
#train.outcome = train.outcome.astype("int32")

In [1928]:
train_bac = train.copy()
train_bac['neutral'] = False
train_bac[['home_team','away_team']]=train_bac[['away_team','home_team']]
train_bac[['home_rating','away_rating']]=train_bac[['away_rating', 'home_rating']]
train_bac[['home_score','away_score']]=train_bac[['away_score', 'home_score']]

In [1929]:
train

,date,home_team,away_team,home_score,away_score,neutral,home_rating,away_rating,temp,prcp,snow,wspd,pres
0,2012-10-12,Antigua and Barbuda,United States,1,2,False,1284,1752,9.4,0.0,NaN,37.2,1011.8
1,2012-10-12,Argentina,Uruguay,3,0,False,1928,1987,16.5,NaN,NaN,NaN,NaN
2,2012-10-12,Belarus,Spain,0,4,False,1585,2126,6.1,2.0,NaN,3.3,NaN
3,2012-10-12,Bulgaria,Denmark,1,1,False,1640,1805,NaN,NaN,NaN,NaN,NaN
4,2012-10-12,Canada,Cuba,3,0,False,1576,1384,6.2,0.8,0.0,16.7,1026.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,2022-06-14,El Salvador,United States,1,1,False,1499,1831,19.1,38.7,NaN,11.1,1012.8
2156,2022-06-14,Chile,Ghana,0,0,True,1768,1551,20.0,15.7,NaN,22.6,1010.4
2157,2022-06-14,Japan,Tunisia,0,3,False,1796,1612,20.0,15.7,NaN,22.6,1010.4
2162,2022-09-23,Japan,United States,2,0,True,1796,1831,13.9,0.0,NaN,12.6,1016.8


In [1930]:
train_bac

,date,home_team,away_team,home_score,away_score,neutral,home_rating,away_rating,temp,prcp,snow,wspd,pres
0,2012-10-12,United States,Antigua and Barbuda,2,1,False,1752,1284,9.4,0.0,NaN,37.2,1011.8
1,2012-10-12,Uruguay,Argentina,0,3,False,1987,1928,16.5,NaN,NaN,NaN,NaN
2,2012-10-12,Spain,Belarus,4,0,False,2126,1585,6.1,2.0,NaN,3.3,NaN
3,2012-10-12,Denmark,Bulgaria,1,1,False,1805,1640,NaN,NaN,NaN,NaN,NaN
4,2012-10-12,Cuba,Canada,0,3,False,1384,1576,6.2,0.8,0.0,16.7,1026.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,2022-06-14,United States,El Salvador,1,1,False,1831,1499,19.1,38.7,NaN,11.1,1012.8
2156,2022-06-14,Ghana,Chile,0,0,False,1551,1768,20.0,15.7,NaN,22.6,1010.4
2157,2022-06-14,Tunisia,Japan,3,0,False,1612,1796,20.0,15.7,NaN,22.6,1010.4
2162,2022-09-23,United States,Japan,0,2,False,1831,1796,13.9,0.0,NaN,12.6,1016.8


In [1931]:
train = pd.concat([train, train_bac])

## Process Predictions set

Yes I should have done this with the rest of the games, but i made a mistake

In [1932]:
resultscup = resultswc.copy()

In [1933]:
from itertools import repeat

In [1934]:
ratings_by_year = [{} for i in repeat(None, 1)]

In [1935]:
years = range(2022, 2023)
for idx, year in enumerate(years):
    url = f'https://www.international-football.net/elo-ratings-table?year={year}&month=06&day=01&confed=&old-team=#selection'
    page = requests.get(url) # Getting page HTML through request
    soup = BeautifulSoup(page.content, 'html.parser') # Parsing content using beautifulsoup
    trim = soup.text[soup.text.find('As calculated on website www.eloratings.net'):]
    for team in all_teams:
        team_name = '.' + team
        counter = 0
        if team_name in trim:
            try:
                ratings_by_year[idx][team] = int(trim[trim.find(team) + len(team): trim.find(team) + len(team) + 4])
            except Exception:
                while counter < 10:
                    updated_trim = trim[trim.find(team) + len(team):]
                    try:
                        ratings_by_year[idx][team] = int(updated_trim[updated_trim.find(team) + len(team): updated_trim.find(team) + len(team) + 4])
                        break
                    except Exception:
                        counter += 1

In [1936]:
#results = results.reset_index()  # make sure indexes pair with number of rows
resultscup['home_rating'] = 0
resultscup['away_rating'] = 0

In [1937]:
resultscup.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rating,away_rating
44104,2022-11-20,Qatar,Ecuador,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False,0,0
44105,2022-11-21,Senegal,Netherlands,NaN,NaN,FIFA World Cup,Doha,Qatar,True,0,0
44106,2022-11-21,England,Iran,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,0,0
44107,2022-11-21,United States,Wales,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,0,0
44108,2022-11-22,Argentina,Saudi Arabia,NaN,NaN,FIFA World Cup,Lusail,Qatar,True,0,0


In [1938]:
resultscup = resultscup.reset_index(drop=True)
resultscup['date'] = pd.to_datetime(resultscup['date'])


for idx, row in resultscup.iterrows():
    item = 0
    try:
        rat = ratings_by_year[item][row.home_team]
        if rat > 3000:
            rat = int(str(rat)[:-1])
        resultscup.home_rating[idx] = rat
    except Exception:
        if row.home_team == 'Macedonia':
            resultscup.home_rating[idx] = 1518
        elif row.home_team == 'United States Virgin Islands':
            resultscup.home_rating[idx] = 680
        elif row.home_team == 'Taiwan':
            resultscup.home_rating[idx] = 950
    try:
        rat = ratings_by_year[item][row.away_team]
        if rat > 3000:
            rat = int(str(rat)[:-1])
        resultscup.away_rating[idx] = rat
    except Exception:
        if row.away_team == 'Macedonia':
            resultscup.home_rating[idx] = 1518
        elif row.away_team == 'United States Virgin Islands':
            resultscup.home_rating[idx] = 680
        elif row.away_team == 'Taiwan':
            resultscup.home_rating[idx] = 950

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1939]:
resultscup.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rating,away_rating
0,2022-11-20,Qatar,Ecuador,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False,1663,1840
1,2022-11-21,Senegal,Netherlands,NaN,NaN,FIFA World Cup,Doha,Qatar,True,1729,1938
2,2022-11-21,England,Iran,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,2039,1820
3,2022-11-21,United States,Wales,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,1831,1825
4,2022-11-22,Argentina,Saudi Arabia,NaN,NaN,FIFA World Cup,Lusail,Qatar,True,2138,1634


In [1940]:
resultscup['long'] = 0.0
resultscup['lat'] = 0.0

In [1941]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

In [1942]:
resultscup = resultscup.reset_index(drop=True)
# introduce parralel programming if run again.
for idx, row in resultscup.iterrows():
    try:
        location = geolocator.geocode(row.city)[-1]
        long =  location[0]
        lat =  location[1]
        resultscup.long[idx] = long
        resultscup.lat[idx] = lat
    except Exception:
        try: 
            location = geolocator.geocode(row.country)[-1]
            long =  location[0]
            lat =  location[1]
            resultscup.long[idx] = long
            resultscup.lat[idx] = lat
        except Exception:
            pass

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1943]:
resultscup.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rating,away_rating,long,lat
0,2022-11-20,Qatar,Ecuador,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False,1663,1840,25.559884,55.564020
1,2022-11-21,Senegal,Netherlands,NaN,NaN,FIFA World Cup,Doha,Qatar,True,1729,1938,25.285633,51.526416
2,2022-11-21,England,Iran,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,2039,1820,24.930474,50.977689
3,2022-11-21,United States,Wales,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True,1831,1825,24.930474,50.977689
4,2022-11-22,Argentina,Saudi Arabia,NaN,NaN,FIFA World Cup,Lusail,Qatar,True,2138,1634,25.422889,51.519716


In [1944]:
resultscup['temp'] = np.nan
resultscup['prcp'] = np.nan
resultscup['snow'] = np.nan
resultscup['wspd'] = np.nan
resultscup['pres'] = np.nan

In [1945]:
resultscup.loc[0]

date           2022-11-20 00:00:00
home_team                    Qatar
away_team                  Ecuador
home_score                     NaN
away_score                     NaN
tournament          FIFA World Cup
city                       Al Khor
country                      Qatar
neutral                      False
home_rating                   1663
away_rating                   1840
long                       25.5599
lat                         55.564
temp                           NaN
prcp                           NaN
snow                           NaN
wspd                           NaN
pres                           NaN
Name: 0, dtype: object

In [1946]:
resultscup.date.loc[(resultscup['date'] >= '2022-11-27')] = '2022-10-27' # Dates after the 27th don't appear to work

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1947]:
resultscup['date'] = pd.to_datetime(resultscup['date'])
pool = mp.Pool(processes=mp.cpu_count())
new_rows = pool.map(multi_task.task, [row for row in resultscup.iterrows()])
weather_resultscup = pd.DataFrame.from_records(new_rows, columns = list(resultscup.keys()))

In [1948]:
test = weather_resultscup.drop(['date','city', 'long', 'lat', 'tournament', 'country'], axis=1)

In [1949]:
test.home_team = test.home_team.astype("category")
test.away_team = test.away_team.astype("category")
#test.country = test.country.astype("category")

test.home_rating = test.home_rating.astype("int32")
test.away_rating = test.away_rating.astype("int32")
#test.home_score = test.home_score.astype("int32")
#test.away_score = test.away_score.astype("int32")
#train.outcome = train.outcome.astype("int32")

In [1950]:
test.dtypes

home_team      category
away_team      category
home_score      float64
away_score      float64
neutral            bool
home_rating       int32
away_rating       int32
temp            float64
prcp            float64
snow            float64
wspd            float64
pres            float64
dtype: object

In [1951]:
test_bac = test.copy()
test_bac['neutral'] = True
test_bac[['home_team','away_team']]=test_bac[['away_team','home_team']]
test_bac[['home_rating','away_rating']]=test_bac[['away_rating', 'home_rating']]
test_bac[['home_score','away_score']]=test_bac[['away_score', 'home_score']]

In [1952]:
test = pd.concat([test, test_bac])

In [1953]:
test

,home_team,away_team,home_score,away_score,neutral,home_rating,away_rating,temp,prcp,snow,wspd,pres
0,Qatar,Ecuador,NaN,NaN,False,1663,1840,26.9,NaN,NaN,12.3,1015.1
1,Senegal,Netherlands,NaN,NaN,True,1729,1938,25.0,NaN,NaN,19.1,1017.7
2,England,Iran,NaN,NaN,True,2039,1820,NaN,NaN,NaN,NaN,NaN
3,United States,Wales,NaN,NaN,True,1831,1825,NaN,NaN,NaN,NaN,NaN
4,Argentina,Saudi Arabia,NaN,NaN,True,2138,1634,25.1,NaN,NaN,12.6,1018.1
...,...,...,...,...,...,...,...,...,...,...,...,...
43,Morocco,Canada,NaN,NaN,True,1729,1798,29.9,0.0,NaN,8.5,1014.3
44,Switzerland,Serbia,NaN,NaN,True,1920,1845,29.9,0.0,NaN,8.5,1014.3
45,Brazil,Cameroon,NaN,NaN,True,2155,1631,29.9,0.0,NaN,10.6,1014.1
46,Uruguay,Ghana,NaN,NaN,True,1923,1551,29.9,0.0,NaN,8.5,1014.3


In [1954]:
test.home_team = test.home_team.astype("category")
test.away_team = test.away_team.astype("category")
#test.country = test.country.astype("category")

test.home_rating = test.home_rating.astype("int32")
test.away_rating = test.away_rating.astype("int32")
#test.home_score = test.home_score.astype("int32")
#test.away_score = test.away_score.astype("int32")
#train.outcome = train.outcome.astype("int32")

In [1955]:
xout = test.drop(['home_score', 'away_score'], axis=1)

dout = xgb.DMatrix(xout, enable_categorical = True)

# XGB SET E

In [1956]:
train.home_team = train.home_team.astype("category")
train.away_team = train.away_team.astype("category")
#test.country = test.country.astype("category")

train.home_rating = train.home_rating.astype("int32")
train.away_rating = train.away_rating.astype("int32")
#test.home_score = test.home_score.astype("int32")
#test.away_score = test.away_score.astype("int32")
#train.outcome = train.outcome.astype("int32")

In [1957]:
hist_train = train.copy()
train = train.drop('date', axis=1)

## Home Team Goals

In [1958]:
X = train.drop(['home_score', 'away_score'], axis=1)
y = train['home_score']

In [1959]:
import xgboost as xgb
dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)

In [1960]:
from sklearn.metrics import mean_absolute_error

In [1961]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [1962]:
params['eval_metric'] = "mae"
num_boost_round = 10000

model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

cv_results['test-mae-mean'].min()

In [1963]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [1964]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[22:27:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [1965]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [1966]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [1967]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[22:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [1968]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [1969]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[22:28:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:28:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [1970]:
params['eta'] = best_params

In [1971]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [1972]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[22:31:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:31:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [1973]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [1974]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [1975]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[22:32:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [1976]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [1977]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[22:32:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [1978]:
params['eta'] = best_params

In [1979]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [1980]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[22:35:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [1981]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [1982]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [1983]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[23:40:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [1984]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [1985]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:07:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:07:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:07:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:07:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:07:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [1986]:
params['eta'] = best_params

In [1987]:
model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=best_boost,
    #evals=[(dtest, "Test")],
    #early_stopping_rounds=10
)


[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Predictions

## Set E

In [1988]:
y_pred_h = model_h.predict(dout)
predictions_h = [round(value) for value in y_pred_h]

In [1989]:
final = xout.copy()

In [1990]:
final['home_goals_E'] = y_pred_h

In [1991]:
final.drop(final.columns.difference(['home_team','away_team', 'home_goals_E']), 1, inplace=True)

In [1992]:
final1 = final[0:48]
final2 = final[48:]
final2.rename(columns={"home_goals_E": "away_goals_E"}, inplace=True)
final2[['home_team','away_team']]=final2[['away_team', 'home_team']]

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1993]:
finalout = pd.merge(final1, final2, how = 'left', left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

# XGB SET D

In [1994]:
hist_train.head()
train = hist_train[(hist_train['date'] > "2016-01-01")]
train = train.drop('date', axis = 1)

## Home Team Goals

In [1995]:
X = train.drop(['home_score', 'away_score'], axis=1)
y = train['home_score']

In [1996]:
from sklearn.model_selection import train_test_split
x_test = y_test = 0
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0, random_state=42)

In [1997]:
import xgboost as xgb
dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
#dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical = True)

In [1998]:
from sklearn.metrics import mean_absolute_error

In [1999]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [2000]:
params['eval_metric'] = "mae"
num_boost_round = 10000

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

cv_results['test-mae-mean'].min()

In [2001]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2002]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:10:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2003]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2004]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2005]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:11:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2006]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2007]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:11:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:11:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2008]:
params['eta'] = best_params

In [2009]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2010]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:13:30] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:13:30] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:13:30] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:13:30] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:13:30] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2011]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2012]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2013]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:14:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2014]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2015]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:14:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:14:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2016]:
params['eta'] = best_params

In [2017]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2018]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:16:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:16:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:16:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:16:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:16:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2019]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2020]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2021]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:17:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2022]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2023]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:17:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:17:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2024]:
params['eta'] = best_params

In [2025]:
model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=best_boost,
    #evals=[(dtest, "Test")],
    #early_stopping_rounds=10
)


[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Predictions

## Set D

In [2026]:
y_pred_h = model_h.predict(dout)
predictions_h = [round(value) for value in y_pred_h]

In [2027]:
final = xout.copy()

In [2028]:
final['home_goals_D'] = y_pred_h

In [2029]:
final.drop(final.columns.difference(['home_team','away_team', 'home_goals_D']), 1, inplace=True)

In [2030]:
final1 = final[0:48]
final2 = final[48:]
final2.rename(columns={"home_goals_D": "away_goals_D"}, inplace=True)
final2[['home_team','away_team']]=final2[['away_team', 'home_team']]

In [2031]:
finalout_temp = pd.merge(final1, final2, how = 'left', left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

In [2032]:
finalout = pd.merge(finalout, finalout_temp, left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

# XGB SET C

In [2033]:
hist_train.head()
train = hist_train[(hist_train['date'] > "2018-01-01")]
train = train.drop('date', axis = 1)

## Home Team Goals

In [2034]:
X = train.drop(['home_score', 'away_score'], axis=1)
y = train['home_score']

In [2035]:
from sklearn.model_selection import train_test_split
x_test = y_test = 0
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0, random_state=42)

In [2036]:
import xgboost as xgb
dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
#dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical = True)

In [2037]:
from sklearn.metrics import mean_absolute_error

In [2038]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [2039]:
params['eval_metric'] = "mae"
num_boost_round = 10000

model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

cv_results['test-mae-mean'].min()

In [2040]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2041]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2042]:
best_params[0]

2

In [2043]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2044]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2045]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:20:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2046]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2047]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
        best_boost = boost_rounds
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:20:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2048]:
params['eta'] = best_params

In [2049]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2050]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:21:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2051]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2052]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2053]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:21:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2054]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2055]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:21:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:21:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2056]:
params['eta'] = best_params

In [2057]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2058]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:23:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2059]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2060]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2061]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:23:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2062]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2063]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:23:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2064]:
params['eta'] = best_params

In [2065]:
model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=best_boost,
    #evals=[(dtest, "Test")],
    #early_stopping_rounds=10
)


[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Predictions

## Set C

In [2066]:
y_pred_h = model_h.predict(dout)
predictions_h = [round(value) for value in y_pred_h]

In [2067]:
final = xout.copy()

In [2068]:
final['home_goals_C'] = y_pred_h

In [2069]:
final.drop(final.columns.difference(['home_team','away_team', 'home_goals_C']), 1, inplace=True)

In [2070]:
final1 = final[0:48]
final2 = final[48:]
final2.rename(columns={"home_goals_C": "away_goals_C"}, inplace=True)
final2[['home_team','away_team']]=final2[['away_team', 'home_team']]

In [2071]:
finalout_temp = pd.merge(final1, final2, how = 'left', left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

In [2072]:
finalout = pd.merge(finalout, finalout_temp, left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

# XGB SET B

In [2073]:
hist_train.head()
train = hist_train[(hist_train['date'] > "2019-01-01")]
train = train.drop('date', axis = 1)

## Home Team Goals

In [2074]:
X = train.drop(['home_score', 'away_score'], axis=1)
y = train['home_score']

In [2075]:
from sklearn.model_selection import train_test_split
x_test = y_test = 0
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0, random_state=42)

In [2076]:
import xgboost as xgb
dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
#dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical = True)

In [2077]:
from sklearn.metrics import mean_absolute_error

In [2078]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [2079]:
params['eval_metric'] = "mae"
num_boost_round = 10000

model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

cv_results['test-mae-mean'].min()

In [2080]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2081]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2082]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2083]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2084]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2085]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2086]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost - boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:24:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:24:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2087]:
params['eta'] = best_params

In [2088]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2089]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:26:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:26:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:26:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:26:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:26:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2090]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2091]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2092]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:27:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2093]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2094]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:27:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:27:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2095]:
params['eta'] = best_params

In [2096]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2097]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:28:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2098]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2099]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2100]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:28:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2101]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2102]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:28:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:28:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2103]:
params['eta'] = best_params

In [2104]:
model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=best_boost,
    #evals=[(dtest, "Test")],
    #early_stopping_rounds=10
)


[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Predictions

## Set B

In [2105]:
y_pred_h = model_h.predict(dout)
predictions_h = [round(value) for value in y_pred_h]

In [2106]:
final = xout.copy()

In [2107]:
final['home_goals_B'] = y_pred_h

In [2108]:
final.drop(final.columns.difference(['home_team','away_team', 'home_goals_B']), 1, inplace=True)

In [2109]:
final1 = final[0:48]
final2 = final[48:]
final2.rename(columns={"home_goals_B": "away_goals_B"}, inplace = True)
final2[['home_team','away_team']]=final2[['away_team', 'home_team']]

In [2110]:
finalout_temp = pd.merge(final1, final2, how = 'left', left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

In [2111]:
finalout = pd.merge(finalout, finalout_temp, left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

# XGB SET A

In [2112]:
hist_train.head()
train = hist_train[(hist_train['date'] > "2021-01-01")]
train = train.drop('date', axis = 1)

## Home Team Goals

In [2113]:
X = train.drop(['home_score', 'away_score'], axis=1)
y = train['home_score']

In [2114]:
from sklearn.model_selection import train_test_split
x_test = y_test = 0
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0, random_state=42)

In [2115]:
import xgboost as xgb
dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
#dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical = True)

In [2116]:
from sklearn.metrics import mean_absolute_error

In [2117]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [2118]:
params['eval_metric'] = "mae"
num_boost_round = 10000

model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

cv_results['test-mae-mean'].min()

In [2119]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2120]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2121]:
best_params[0]

4

In [2122]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2123]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2124]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:30:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2125]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2126]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:30:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:30:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2127]:
params['eta'] = best_params

In [2128]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2129]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:31:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:31:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:31:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:31:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:31:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2130]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2131]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2132]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:32:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2133]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2134]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:32:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:32:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2135]:
params['eta'] = best_params

In [2136]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,12)
    for min_child_weight in range(1,8)
]

In [2137]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=2, min_child_weight=1
[00:33:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:33:35] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-

In [2138]:
# Best from above
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [2139]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,11)]
    for colsample in [i/10. for i in range(5,11)]
]

In [2140]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[00:34:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpy

In [2141]:
params['subsample'] =  best_params[0]
params['colsample_bytree'] =  best_params[1]

In [2142]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, 0.25, .01, .005, .0025, .0001, .0005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_boost = boost_rounds
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[00:34:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[00:34:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/o

In [2143]:
params['eta'] = best_params

In [2144]:
model_h = xgb.train(
    params,
    dtrain,
    num_boost_round=best_boost,
    #evals=[(dtest, "Test")],
    #early_stopping_rounds=10
)


[00:35:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Predictions

## Set A

In [2145]:
y_pred_h = model_h.predict(dout)
predictions_h = [round(value) for value in y_pred_h]

In [2146]:
final = xout.copy()

In [2147]:
final['home_goals_A'] = y_pred_h

In [2148]:
final.drop(final.columns.difference(['home_team','away_team', 'home_goals_A']), 1, inplace=True)

In [2149]:
final1 = final[0:48]
final2 = final[48:]
final2.rename(columns={"home_goals_A": "away_goals_A"}, inplace=True)
final2[['home_team','away_team']]=final2[['away_team', 'home_team']]

In [2150]:
finalout_temp = pd.merge(final1, final2, how = 'left', left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

In [2151]:
finalout = pd.merge(finalout, finalout_temp, left_on = ['home_team', 'away_team'], right_on = ['home_team', 'away_team'])

In [2152]:
finalout

,home_team,away_team,home_goals_E,away_goals_E,home_goals_D,away_goals_D,home_goals_C,away_goals_C,home_goals_B,away_goals_B,home_goals_A,away_goals_A
0,Qatar,Ecuador,0.796737,1.552365,0.887463,1.501480,0.697166,1.207196,0.704832,0.918751,0.557307,1.265603
1,Senegal,Netherlands,0.751341,1.620837,0.840815,1.703799,0.642924,1.761794,0.768031,1.559760,0.699661,1.343311
2,England,Iran,1.451221,0.800959,1.238666,0.659180,1.629864,0.562886,1.122840,0.647971,1.054264,0.527656
3,United States,Wales,1.005368,1.005368,0.967167,0.907476,0.911486,0.911486,0.972232,0.851895,0.798141,0.669542
4,Argentina,Saudi Arabia,2.209063,0.617822,2.373329,0.529573,2.124413,0.233056,2.157148,0.614753,1.874352,0.457600
5,Mexico,Poland,0.960867,0.933040,1.173691,1.023911,0.965027,0.773957,0.925932,0.741455,0.956121,0.599462
6,Denmark,Tunisia,1.815993,0.648203,1.645757,0.542180,2.002227,0.493671,2.050033,0.502380,1.272244,0.407139
7,France,Australia,2.075378,0.701191,2.007719,0.481705,1.830439,0.485394,1.622731,0.614753,1.860981,0.485129
8,Germany,Japan,1.396860,0.868195,1.184638,0.719177,1.629864,0.720417,1.122840,0.647971,1.045592,0.517691
9,Spain,Costa Rica,1.667718,0.696667,1.685884,0.609682,1.830439,0.485394,1.622731,0.721683,1.570826,0.526551


In [2153]:
finalout.to_csv('../dat/finalout.csv', index=False)

In [2190]:
weights = [1,1,2,2,2]
finalout['home_ex_goals'] = (weights[0] * finalout['home_goals_E'] + weights[1] * finalout['home_goals_D'] +  weights[2] * finalout['home_goals_C'] + weights[3] * finalout['home_goals_B'] + weights [4] * finalout['home_goals_A']) / sum(weights)
finalout['away_ex_goals'] = (weights[0] * finalout['away_goals_E'] + weights[1] * finalout['away_goals_D'] +  weights[2] * finalout['away_goals_C'] + weights[3] * finalout['away_goals_B'] + weights [4] * finalout['away_goals_A']) / sum(weights)

In [2191]:
pred_goals = finalout.copy()
pred_goals.drop(pred_goals.columns.difference(['home_team','away_team', 'home_ex_goals', 'away_ex_goals']), 1, inplace=True)
pred_goals['Home_score'] = np.nan
pred_goals['Away_score'] = np.nan
pred_goals['Home_win_chance'] = np.nan
pred_goals['Draw_chance'] = np.nan
pred_goals['Away_win_chance'] = np.nan

In [2192]:
%%capture
s = 1000000

for idx, row in pred_goals.iterrows():
    t1 = np.random.poisson(lam=row[2], size=s)
    t2 = np.random.poisson(lam=row[3], size=s)
    df = pd.DataFrame({'t1':t1, 't2':t2})
    df.value_counts(["t1", "t2"])
    hwc = sum(df.t1 > df.t2) / s
    dc = sum(df.t1 == df.t2) / s
    awc = sum(df.t1 < df.t2) / s
   # if hwc > 0.3 and awc > 0.3:
    #    home = pd.DataFrame(df.loc[df['t1'] == df['t2']].value_counts(["t1", "t2"])).index[0][0]
     #   away = pd.DataFrame(df.loc[df['t1'] == df['t2']].value_counts(["t1", "t2"])).index[0][1]
    if hwc > dc and hwc > awc:
        home = pd.DataFrame(df.loc[df['t1'] > df['t2']].value_counts(["t1", "t2"])).index[0][0]
        away = pd.DataFrame(df.loc[df['t1'] > df['t2']].value_counts(["t1", "t2"])).index[0][1]
    elif dc > hwc and dc > awc:
        home = pd.DataFrame(df.loc[df['t1'] == df['t2']].value_counts(["t1", "t2"])).index[0][0]
        away = pd.DataFrame(df.loc[df['t1'] == df['t2']].value_counts(["t1", "t2"])).index[0][1]
    elif awc > hwc and awc > dc:
        home = pd.DataFrame(df.loc[df['t1'] < df['t2']].value_counts(["t1", "t2"])).index[0][0]
        away = pd.DataFrame(df.loc[df['t1'] < df['t2']].value_counts(["t1", "t2"])).index[0][1]
    pred_goals.Home_score[idx] = home
    pred_goals.Away_score[idx] = away
    pred_goals.Home_win_chance[idx] = hwc
    pred_goals.Draw_chance[idx] = dc
    pred_goals.Away_win_chance[idx] = awc
    

In [2193]:
pred_goals

,home_team,away_team,home_ex_goals,away_ex_goals,Home_score,Away_score,Home_win_chance,Draw_chance,Away_win_chance
0,Qatar,Ecuador,0.700351,1.229618,0.0,1.0,0.212005,0.299771,0.488224
1,Senegal,Netherlands,0.726674,1.581796,0.0,1.0,0.170954,0.251186,0.577860
2,England,Iran,1.287978,0.617145,1.0,0.0,0.529069,0.292588,0.178343
3,United States,Wales,0.917032,0.847336,1.0,0.0,0.353334,0.332388,0.314278
4,Argentina,Saudi Arabia,2.111777,0.469777,2.0,0.0,0.759205,0.171018,0.069777
5,Mexico,Poland,0.978590,0.773337,1.0,0.0,0.390782,0.331900,0.277318
6,Denmark,Tunisia,1.763845,0.499596,1.0,0.0,0.683316,0.217447,0.099237
7,France,Australia,1.838925,0.544181,1.0,0.0,0.686019,0.211093,0.102888
8,Germany,Japan,1.272261,0.669941,1.0,0.0,0.509773,0.294260,0.195967
9,Spain,Costa Rica,1.675199,0.596701,1.0,0.0,0.636547,0.234448,0.129005


In [2195]:
pred_goals.home_ex_goals.mean() + pred_goals.away_ex_goals.mean()

2.1373958587646484

In [2158]:
gal = ['Qatar', 'Ecuador', 'Senegal', 'Netherlands']
gbl = ['England', 'Iran', 'United States', 'Wales']
gcl = ['Argentina', 'Saudi Arabia', 'Mexico', 'Poland']
gdl = ['France', 'Australia', 'Denmark', 'Tunisia']
gel = ['Spain', 'Costa Rica', 'Germany', 'Japan']
gfl = ['Belgium', 'Morocco', 'Croatia', 'Canada']
ggl = ['Brazil', 'Switzerland', 'Serbia', 'Cameroon']
ghl = ['Portugal', 'Uruguay', 'South Korea', 'Ghana']

In [2159]:
ga = pd.DataFrame({'Team':gal, 'Score':0, 'Draw':0})
gb = pd.DataFrame({'Team':gbl, 'Score':0, 'Draw':0})
gc = pd.DataFrame({'Team':gcl, 'Score':0, 'Draw':0})
gd = pd.DataFrame({'Team':gdl, 'Score':0, 'Draw':0})
ge = pd.DataFrame({'Team':gel, 'Score':0, 'Draw':0})
gf = pd.DataFrame({'Team':gfl, 'Score':0, 'Draw':0})
gg = pd.DataFrame({'Team':ggl, 'Score':0, 'Draw':0})
gh = pd.DataFrame({'Team':ghl, 'Score':0, 'Draw':0})

In [2160]:
pred_tourn = pred_goals.copy()

In [2161]:
for idx, row in enumerate(pred_tourn.keys()):
    print(idx, row)

0 home_team
1 away_team
2 home_ex_goals
3 away_ex_goals
4 Home_score
5 Away_score
6 Home_win_chance
7 Draw_chance
8 Away_win_chance


In [2162]:
 ga.Score.loc[ga['Team'] == 'Qatar'] 

0    0
Name: Score, dtype: int64

In [2163]:
for idx, row in pred_tourn.iterrows():
    if row[0] in gal:
        if  row[4] > row[5]:
            ga.Score.loc[ga['Team'] == row[0]] += 3
            ga.Draw.loc[ga['Team'] == row[0]] += row[7]
            print(row[0], row[1])
        elif row[5] > row[4]:
            ga.Score.loc[ga['Team'] == row[1]] += 3
            ga.Draw.loc[ga['Team'] == row[1]] += row[7]
    elif row[0] in gbl:
        if  row[4] > row[5]:
            gb.Score.loc[gb['Team'] == row[0]] += 3
            gb.Draw.loc[gb['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gb.Score.loc[gb['Team'] == row[1]] += 3
            gb.Draw.loc[gb['Team'] == row[1]] += row[7]
    elif row[0] in gcl:
        if  row[4] > row[5]:
            gc.Score.loc[gc['Team'] == row[0]] += 3
            gc.Draw.loc[gc['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gc.Score.loc[gc['Team'] == row[1]] += 3
            gc.Draw.loc[gc['Team'] == row[1]] += row[7]
    elif row[0] in gdl:
        if  row[4] > row[5]:
            gd.Score.loc[gd['Team'] == row[0]] += 3
            gd.Draw.loc[gd['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gd.Score.loc[gd['Team'] == row[1]] += 3
            gd.Draw.loc[gd['Team'] == row[1]] += row[7]
    elif row[0] in gel:
        if  row[4] > row[5]:
            ge.Score.loc[ge['Team'] == row[0]] += 3
            ge.Draw.loc[ge['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            ge.Score.loc[ge['Team'] == row[1]] += 3
            ge.Draw.loc[ge['Team'] == row[1]] += row[7]
    elif row[0] in gfl:
        if  row[4] > row[5]:
            gf.Score.loc[gf['Team'] == row[0]] += 3
            gf.Draw.loc[gf['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gf.Score.loc[gf['Team'] == row[1]] += 3
            gf.Draw.loc[gf['Team'] == row[1]] += row[7]
    elif row[0] in ggl:
        if  row[4] > row[5]:
            gg.Score.loc[gg['Team'] == row[0]] += 3
            gg.Draw.loc[gg['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gg.Score.loc[gg['Team'] == row[1]] += 3
            gg.Draw.loc[gg['Team'] == row[1]] += row[7]
    elif row[0] in ghl:
        if  row[4] > row[5]:
            gh.Score.loc[gh['Team'] == row[0]] += 3
            gh.Draw.loc[gh['Team'] == row[0]] += row[7]
        elif row[5] > row[4]:
            gh.Score.loc[gh['Team'] == row[1]] += 3
            gh.Draw.loc[gh['Team'] == row[1]] += row[7]
            
ga.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gb.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gc.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gd.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
ge.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gf.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gg.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)
gh.sort_values(["Score", "Draw"],axis = 0, ascending = [False, True], inplace = True)

Netherlands Ecuador
Ecuador Senegal


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2164]:
ga

,Team,Score,Draw
3,Netherlands,9,0.806691
1,Ecuador,6,0.597125
2,Senegal,3,0.323494
0,Qatar,0,0.000000


In [2165]:
gb

,Team,Score,Draw
0,England,9,0.818466
2,United States,6,0.650866
1,Iran,3,0.328867
3,Wales,0,0.000000


In [2166]:
gc

,Team,Score,Draw
0,Argentina,9,0.730621
2,Mexico,6,0.630150
3,Poland,3,0.308020
1,Saudi Arabia,0,0.000000


In [2167]:
gd

,Team,Score,Draw
0,France,9,0.681768
2,Denmark,6,0.473176
1,Australia,3,0.342550
3,Tunisia,0,0.000000


In [2168]:
ge

,Team,Score,Draw
0,Spain,9,0.775416
2,Germany,6,0.540588
3,Japan,3,0.295039
1,Costa Rica,0,0.000000


In [2169]:
gf

,Team,Score,Draw
0,Belgium,9,0.787926
2,Croatia,6,0.622237
3,Canada,3,0.296584
1,Morocco,0,0.000000


In [2170]:
gg

,Team,Score,Draw
0,Brazil,9,0.720964
1,Switzerland,6,0.577022
2,Serbia,3,0.285998
3,Cameroon,0,0.000000


In [2171]:
gh

,Team,Score,Draw
0,Portugal,9,0.655816
1,Uruguay,6,0.478594
2,South Korea,3,0.222144
3,Ghana,0,0.000000
